In [7]:
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import math
import time

import psycopg2
from sqlalchemy import create_engine, Column, Integer, String, LargeBinary, Float
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

from deepface import DeepFace
from deepface.commons import functions

Base = declarative_base()

# create an engine to connect to the PostgreSQL database
engine = create_engine('postgresql://postgres:@localhost:5430/faceio')

Directory  /Users/ds_learner16 /.deepface created
Directory  /Users/ds_learner16 /.deepface/weights created


In [8]:
class FaceMeta(Base):
    __tablename__ = 'face_meta'

    id = Column(Integer, primary_key=True)
    empl_no = Column(Integer, nullable=False)
    img_name = Column(String, nullable=False)
    embedding = Column(LargeBinary, nullable=False)

class FaceEmbedding(Base):
    __tablename__ = 'face_embeddings'

    id = Column(Integer, primary_key=True)
    face_id = Column(Integer, nullable=False)
    dimension = Column(Integer, nullable=False)
    value = Column(Float, nullable=False)


In [10]:
from keras.preprocessing import image
from keras_applications.inception_resnet_v2 import preprocess_input
import tensorflow as tf 

Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()
model = DeepFace.build_model("Facenet")


facial_img_paths = []

for root, directory, files in os.walk("/Users/ds_learner16/Documents/Face.io/photos/train"):
    for file in files:
        if '.jpg' in file:
            facial_img_paths.append(root+"/"+file)
instances = []

for i in tqdm(range(0, len(facial_img_paths))):
    facial_img_path = facial_img_paths[i]

    #load image and preprocess
    img = tf.keras.preprocessing.image.load_img(facial_img_path, target_size=(160, 160))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    #represent
    embedding = model.predict(x)[0]

    #store
    instance = []
    instance.append(facial_img_path)
    instance.append(embedding)
    instances.append(instance)

df = pd.DataFrame(instances, columns=["img_name", "embedding"])

  0%|          | 0/1304 [00:00<?, ?it/s]


AttributeError: module 'keras.preprocessing.image' has no attribute 'load_img'

In [ ]:
for index, instance in tqdm(df.iterrows(), total=df.shape[0]):
    img_name = instance["img_name"]
    embeddings = instance["embedding"]

    face_meta = FaceMeta(empl_no=index, img_name=img_name, embedding=embeddings.tobytes())
    session.add(face_meta)

    for i, embedding in enumerate(embeddings):
        face_embedding = FaceEmbedding(face_id=index, dimension=i, value=float(embedding))
        session.add(face_embedding)

session.commit()


In [4]:
!pip install --no-deps deepface

  Using cached deepface-0.0.79-py3-none-any.whl (49 kB)


In [6]:
!pip install gdown

  Using cached gdown-4.7.1-py3-none-any.whl (15 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
deepface 0.0.79 requires fire>=0.4.0, which is not installed.
deepface 0.0.79 requires gunicorn>=20.1.0, which is not installed.
deepface 0.0.79 requires mtcnn>=0.1.0, which is not installed.
deepface 0.0.79 requires retina-face>=0.0.1, which is not installed.
deepface 0.0.79 requires tensorflow>=1.9.0, which is not installed.
